## Testes Baseados em Estatística Descritiva
Nas próximas aulas veremos diferentes modelos de previsão como ARMA, ARIMA, ARIMA sazonal entre outros. Cada modelo aborda um tipo diferente de série temporal. Por esta razão, para selecionar um modelo apropriado precisamos entender melhor os dados. A estatística descritiva serve exatamente para isso.

Nesta aula aprenderemos como determinar se uma série temporal é *estacionária*, se é *independente* e se duas séries demonstram *correlação* e/ou *causalidade*.

Vamos começar fazendo os imports e carregando os datasets que serão utilizados.

In [2]:
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import adfuller, grangercausalitytests
from statsmodels.tools.eval_measures import mse, rmse, meanabs
%matplotlib inline

# Ignorar warnings 
import warnings
warnings.filterwarnings("ignore")

# Load a seasonal dataset
df1 = pd.read_csv('airline_passengers.csv',index_col='Month',parse_dates=True)
df1.index.freq = 'MS'

# Load a nonseasonal dataset
df2 = pd.read_csv('DailyTotalFemaleBirths.csv',index_col='Date',parse_dates=True)
df2.index.freq = 'D'

### Teste de Estacionariedade
Uma série temporal é <em>estacionária</em> se a média e a variância são fixas entre quaisquer dois pontos equidistantes. Ou seja, não importa onde você faça suas observações, os resultados deverão ser os mesmos. Uma série temporal que mostra sazonalidade <em>não</em> é estacionária.

Um teste de estacionariedade geralmente envolve um teste de hipótese de <a href='https://en.wikipedia.org/wiki/Unit_root_test'>raiz unitária</a>, onde a hipótese nula $H_0$ é que a série é <em >não estacionária</em> e contém uma raiz unitária. A hipótese alternativa $H_1$ suporta a estacionariedade. O <a href='https://en.wikipedia.org/wiki/Augmented_Dickey-Fuller_test'>Dikey-Fuller aumentado</a> e o <a href='https://en.wikipedia.org/wiki/KPSS_test Os testes '>Kwiatkowski-Phillips-Schmidt-Shin</a> são testes de estacionariedade.

### Teste Dikey-Fuller Aumentado
Para determinar se uma série é estacionária, podemos usar o <a href='https://en.wikipedia.org/wiki/Augmented_Dickey-Fuller_test'>teste Dickey-Fuller aumentado</a>. Neste teste, a hipótese nula afirma que $\phi = 1$ (isso também é chamado de teste de unidade). O teste retorna diversas estatísticas que veremos em instantes. Nosso foco está no *p value*. Um *p value* pequeno ($p<0,05$) indica forte evidência contra a hipótese nula.

Para demonstrar, usaremos um conjunto de dados que sabemos que <em>não</em> é estacionário (airline passengers). Primeiro, vamos representar graficamente os dados junto com uma média móvel e desvio padrão de 12 meses:

In [24]:
df1['12-month-MMS'] = df1['Thousands of Passengers'].rolling(window=12).mean()
df1['12-month-DP'] = df1['Thousands of Passengers'].rolling(window=12).std()

df1[['Thousands of Passengers','12-month-MMS','12-month-DP']].plot();

Observamos que este conjunto de dados não só é sazonal com uma clara tendência ascendente, como também o desvio padrão aumenta ao longo do tempo.

Vamos fazer o teste de Dickey-Fuller aumentado e observar os resultados.

In [25]:
print('Teste Dickey-Fuller aumentado para o dataset airline passengers')
dftest = adfuller(df1['Thousands of Passengers'],autolag='AIC')
dftest

Teste Dickey-Fuller aumentado para o dataset airline passengers


(0.8153688792060597,
 0.9918802434376411,
 13,
 130,
 {'1%': -3.4816817173418295,
  '5%': -2.8840418343195267,
  '10%': -2.578770059171598},
 996.692930839019)

Vamos acrescentar legendas para ficar mais fácil de interpretar os resultados do teste.

In [26]:
print('Teste Dickey-Fuller aumentado para o dataset airline passengers')

dfout = pd.Series(dftest[0:4],index=['ADF estatística de teste','p-value','# lags usedos','# observações'])

for key,val in dftest[4].items():
    dfout[f'Valor crítico ({key})']=val
print(dfout)

Teste Dickey-Fuller aumentado para o dataset airline passengers
ADF estatística de teste      0.815369
p-value                       0.991880
# lags usedos                13.000000
# observações               130.000000
Valor crítico (1%)           -3.481682
Valor crítico (5%)           -2.884042
Valor crítico (10%)          -2.578770
dtype: float64


Aqui temos um p-value muito alto de 0,99, que fornece evidências fracas contra a hipótese nula e, portanto, <em>não rejeitamos</em> a hipótese nula e decidimos que nosso conjunto de dados não é estacionário. \
Nota: em estatística, ao não “aceitarmos” uma hipótese nula, nada é verdadeiramente provado, apenas falhamos em rejeitá-la. \
Agora vamos aplicar o teste ADF a dados estacionários com o conjunto de dados Daily Total Female Births.

In [27]:
df2['30-Day-MMS'] = df2['Births'].rolling(window=30).mean()
df2['30-Day-DP'] = df2['Births'].rolling(window=30).std()

df2[['Births','30-Day-MMS','30-Day-DP']].plot();

In [28]:
print('Teste Dickey-Fuller aumentado para o dataset Daily Female Births')
dftest = adfuller(df2['Births'],autolag='AIC')
dfout = pd.Series(dftest[0:4],index=['Estatística de teste ADF','p-value','# lags usados','# observações'])

for key,val in dftest[4].items():
    dfout[f'Valor crítico ({key})']=val
print(dfout)

Teste Dickey-Fuller aumentado para o dataset Daily Female Births
Estatística de teste ADF     -4.808291
p-value                       0.000052
# lags usados                 6.000000
# observações               358.000000
Valor crítico (1%)           -3.448749
Valor crítico (5%)           -2.869647
Valor crítico (10%)          -2.571089
dtype: float64


Neste caso, o p-value é muito baixo: 0,000052; o que nos leva a rejeitar a hipótese nula. Este conjunto de dados parece não ter raiz unitária e é estacionário.

### Função para facilitar a tomada de decisão baseada no teste ADF
Como o usaremos com frequência nas próximas previsões, vamos definir uma função que possamos copiar em notebooks futuros para executar o teste Dickey-Fuller aumentado. Lembre-se que ainda teremos que importar <tt>adfuller</tt> no topo do nosso notebook.

In [29]:
from statsmodels.tsa.stattools import adfuller

def adf_test(series,title=''):
    """
    Param: série temporal e um título opcional, retorna um relatório ADF

    """
    print(f'Teste Dickey-Fuller Aumentado: {title}')
    result = adfuller(series.dropna(),autolag='AIC') 
    
    labels = ['Estatística de teste ADF','p-value','# lags usados','# observações']
    out = pd.Series(result[0:4],index=labels)

    for key,val in result[4].items():
        out[f'Valor crítico ({key})']=val
        
    print(out.to_string())          # .to_string() remove a linha "dtype: float64"
    
    if result[1] <= 0.05:
        print("Evidência forte contra a hipótese nula.")
        print("Rejeitar a hipótese nula.")
        print("Dados não tem raiz unitária. Série estacionária.")
    else:
        print("Evidência fraca contra a hipótese nula.")
        print("Falha em rejeitar a hipótese nula.")
        print("Dados tem uma raiz unitária. Série não estacionária.")

In [30]:
adf_test(df1['Thousands of Passengers'], 'Airline Passengers')
print()
adf_test(df2['Births'], 'Daily Female Births')

Teste Dickey-Fuller Aumentado: Airline Passengers
Estatística de teste ADF      0.815369
p-value                       0.991880
# lags usados                13.000000
# observações               130.000000
Valor crítico (1%)           -3.481682
Valor crítico (5%)           -2.884042
Valor crítico (10%)          -2.578770
Evidência fraca contra a hipótese nula.
Falha em rejeitar a hipótese nula.
Dados tem uma raiz unitária. Série não estacionária.

Teste Dickey-Fuller Aumentado: Daily Female Births
Estatística de teste ADF     -4.808291
p-value                       0.000052
# lags usados                 6.000000
# observações               358.000000
Valor crítico (1%)           -3.448749
Valor crítico (5%)           -2.869647
Valor crítico (10%)          -2.571089
Evidência forte contra a hipótese nula.
Rejeitar a hipótese nula.
Dados não tem raiz unitária. Série estacionária.


### Teste de Causalidade de Granger
O <a href='https://en.wikipedia.org/wiki/Granger_causality'>Teste de Causalidade de Granger</a> é um teste de hipótese para determinar se uma série temporal é útil na previsão de outra. Embora seja bastante fácil medir correlações entre séries - quando uma sobe, a outra sobe e vice-versa - outra coisa é observar mudanças em uma série correlacionadas com mudanças em outra após um período de tempo consistente. Isto pode indicar a presença de causalidade, de que mudanças na primeira série influenciaram o comportamento da segunda. Contudo, também pode acontecer que ambas as séries sejam afetadas por algum terceiro fator, apenas a taxas diferentes. Ainda assim, pode ser útil se as mudanças numa série puderem prever mudanças futuras noutra, quer haja causalidade ou não. Neste caso, dizemos que uma série “causa Granger” outra.

No caso de duas séries, $y$ e $x$, a hipótese nula é que valores defasados ​​de $x$ <em>não</em> explicam variações em $y$.<br>
Em outras palavras, assume que $x_t$ não causa Granger $y_t$.

Neste exemplo, usaremos o arquivo samples.csv, onde as colunas 'a' e 'd' são conjuntos de dados estacionários.

In [31]:
df3 = pd.read_csv('samples.csv',index_col=0,parse_dates=True)
df3.index.freq = 'MS'
df3.head()

,a,b,c,d
1950-01-01,36,27,0,67
1950-02-01,58,22,3,31
1950-03-01,61,17,5,67
1950-04-01,37,15,8,47
1950-05-01,66,13,8,62


É difícil observar a partir da figura, mas <tt>df['d']</tt> prevê quase perfeitamente o comportamento de <tt>df['a']</tt>.<br>
Para ver isso mais claramente, avançaremos <tt>df['d']</tt> dois períodos.

Vamos agora executar o teste. \
A função recebe um array 2D [y,x] e um número máximo de defasagens (lags) para testar em x. Aqui nosso y é a coluna 'a' e x é a coluna 'd'. Usaremos maxlags = 3.

Valores extremamente baixos de p, como vemos no lag 2, apontam a causalidade. \
Em seguida, vamos comparar dois conjuntos de dados que não são nada semelhantes, 'b' e 'd'.

Aqui não temos valores de p tendendo a zero, o que mostra que não há causalidade nessas duas séries temporais.

### Avaliação de Precisão
Dois cálculos relacionados à regressão linear são <a href='https://en.wikipedia.org/wiki/Mean_squared_error'><strong>erro quadrático médio</strong></a> (MSE) e <a href=' https://en.wikipedia.org/wiki/Root-mean-square_deviation'><strong>raiz do erro quadrático médio</strong></a> (RMSE).

A fórmula do erro quadrático médio é <br><br>
&nbsp;&nbsp;&nbsp;&nbsp;$RMSE = \sqrt{MSE} = \sqrt{{\frac 1 L} \sum\limits_{l=1}^L (y_{T+l} - \hat y_{T+l})^2}$<br><br>
A vantagem do RMSE é que ele é expresso nas mesmas unidades que os dados. <br><br>

Um método semelhante ao RMSE é o <a href='https://en.wikipedia.org/wiki/Mean_absolute_error'><strong>erro absoluto médio</strong></a> (MAE), que é a média das magnitudes do erro, dadas como<br><br>

&nbsp;&nbsp;&nbsp;&nbsp;$MAE = {\frac 1 L} \sum\limits_{l=1}^L \mid{y_{T+l}} - \hat y_{T+l}\mid$<br><br>

Um método de previsão que minimize o MAE levará a previsões da mediana, enquanto a minimização do RMSE levará a previsões da média.

In [32]:
import numpy as np
import pandas as pd
%matplotlib inline
np.random.seed(42)
df = pd.DataFrame(np.random.randint(20,30,(50,2)),columns=['teste','previsão'])
df.plot(figsize=(12,4));

In [33]:
MSE = mse(df['teste'],df['previsão'])
RMSE = rmse(df['teste'],df['previsão'])
MAE = meanabs(df['teste'],df['previsão'])

print(f'Modelo  MSE: {MSE:.3f}')
print(f'Modelo RMSE: {RMSE:.3f}')
print(f'Modelo  MAE: {MAE:.3f}')

Modelo  MSE: 17.020
Modelo RMSE: 4.126
Modelo  MAE: 3.540


### AIC / BIC
Testes mais sofisticados incluem o <a href='https://en.wikipedia.org/wiki/Akaike_information_criterion'><strong>critério de informação Akaike</strong></a> (AIC) e o <a href='https ://en.wikipedia.org/wiki/Bayesian_information_criterion'><strong>Critério de informação bayesiano</strong></a> (BIC).

A AIC avalia uma coleção de modelos e estima a qualidade de cada modelo em relação aos demais. Penalidades são fornecidas para o número de parâmetros usados ​​em um esforço para impedir o sobreajuste. Quanto mais baixos o AIC e o BIC, melhor deverá ser o modelo na previsão.

Estas funções estão disponíveis como:

&nbsp;&nbsp;&nbsp;&nbsp;<tt>from from statsmodels.tools.eval_measures import aic, bic</tt>

mas raramente os computamos sozinhos, pois eles estão integrados em muitas das ferramentas de modelos estatísticos que usamos.

### Expondo sazonalidade utilizando gráficos mensais e quadrimestrais
Statsmodels possui duas funções de plotagem que agrupam os dados por mês e por trimestre. Observe que se os dados aparecerem como meses, você deverá empregar <em>reamostragem</em> com uma função agregada antes de executar um gráfico trimestral. Esses gráficos retornam um objeto <tt>matplotlib.Figure</tt>.
<h3>Métodos de plotagem relacionados:</h3>
<tt><forte>
<a href='https://www.statsmodels.org/stable/generated/statsmodels.graphics.tsaplots.month_plot.html'>tsaplots.month_plot</a></strong><font color=black>(x) </font>&nbsp;&nbsp;&nbsp;&nbsp;Gráfico sazonal de dados mensais<br>
<forte>
<a href='https://www.statsmodels.org/stable/generated/statsmodels.graphics.tsaplots.quarter_plot.html'>tsaplots.quarter_plot</a></strong><font color=black>(x) </font>&nbsp;&nbsp;Gráfico sazonal de dados trimestrais</tt>

In [34]:
import pandas as pd
import numpy as np
%matplotlib inline

df = pd.read_csv('airline_passengers.csv',index_col='Month',parse_dates=True)
df.index.freq = 'MS'
df.head()

,Thousands of Passengers
Month,
1949-01-01,112
1949-02-01,118
1949-03-01,132
1949-04-01,129
1949-05-01,121


Vamos comparar com nosso conjunto de dados <tt>macrodata.csv</tt> não sazonal:


### Exercício 01
Considere a série temporal contida em "exercicio01.xlsx". Plote o gráfico correspondente e informe se a série é ou não estacionária.

### Exercício 02
Considere a série temporal contida em "exercicio02.xlsx". Plote o gráfico correspondente e informe se a série é ou não estacionária.

### Exercício 03
Neste exercício você deve utilizar o dataset "chicken_egg", que mostra a quantidade de galinhas e ovos produzidos entre os anos de 1930 e 1983 numa dada localidade.

A) Verifique se o número de ovos pode ser usado para prever o número de galinhas no futuro. (Ovos Granger Cause Galinhas).

B) Verifique se o número de galinhas pode ser usado para prever o número de ovos no futuro. (Galinhas Granger Cause Ovos).

In [35]:
#define URL where dataset is located
url = "https://raw.githubusercontent.com/Statology/Miscellaneous/main/chicken_egg.txt"

#read in dataset as pandas DataFrame
df_ex3 = pd.read_csv(url, sep="  ", index_col=0, parse_dates=True)

df_ex3.head()

,chicken,egg
year,,
1930-01-01,468491,3581
1931-01-01,449743,3532
1932-01-01,436815,3327
1933-01-01,444523,3255
1934-01-01,433937,3156


Parte A \
H0: a quantidade de ovos NÃO está relacionada com a quantidade de galinhas no futuro \
H1: a quantidade de ovos está relacionada com a quantidade de galinhas no futuro

Em outras palavras: a quantidade atual de ovos pode ser usada para prever a quantidade futura de galinhas?

Parte B \
H0: a quantidade de galinhas NÃO está relacionada com a quantidade de ovos no futuro \
H1: a quantidade de galinhas está relacionada com a quantidade de ovos no futuro

Em outras palavras: a quantidade atual de galinhas pode ser usada para prever a quantidade futura de ovos?